In [84]:
import pandas as pd
import os, re
from pathlib import Path
import numpy
import requests
import slugify
from dateutil.parser import *
import numpy as np

In [85]:
prefix = """@prefix  rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix  rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
@prefix  owl: <http://www.w3.org/2002/07/owl#>.
@prefix  xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix  foaf: <http://xmlns.com/foaf/0.1/>.
@prefix  halalv: <http://halal.addi.is.its.ac.id/halalv.ttl#>.
@prefix  halalf: <http://halal.addi.is.its.ac.id/foodproducts/>.
@prefix  halali: <http://halal.addi.is.its.ac.id/ingredients/>.
@prefix  halalc: <http://halal.addi.is.its.ac.id/certificates/>.
@prefix  halals: <http://halal.addi.is.its.ac.id/sources/>.
@prefix  halalm: <http://halal.addi.is.its.ac.id/manufactures/>.
@prefix  food: <http://purl.org/foodontology#> .
@prefix  foodlirmm: <http://data.lirmm.fr/ontologies/food#> .
@prefix  gr: <http://purl.org/goodrelations/v1#> . 
@prefix  dbr: <http://dbpedia.org/resource/> .

"""

In [86]:
def formatDateX(string):
    try:
        date = parse(string)
        date = date.strftime('%Y-%m-%d')
        return str(date)
    except:
        return "0000-00-00"

In [87]:
def makeSlug(string):
    slug = slugify.slugify(string)
    slug = slug.replace("-", " ")
    slug = slug.title()
    slug = slug.replace(" ", "_")
    return slug

In [88]:
def getManufactureName(string):
    print("Value string manufacture: " + str(string))
    try:
        product = str(string).split(",", 1)[0].title()
    except:
        product = str(string).title()
    print("Value now string manufacture: " + product)
    return product

In [89]:
def getProductName(string):
    print("Value string product: " + str(string))
    try:
        product = str(string).split(",", 1)[1].title()
    except:
        product = str(string).title()
    print("Value now string product: " + product)
    return product

In [90]:
def getIngredients(string):
    ingredients = str(string).title()
    return ingredients

In [91]:
def getCertTrim(string):
    cert = str(string).strip()
    return cert

In [92]:
def getTrimAndTitle(string):
    newstring = str(string).strip()
    newstring = newstring.title()
    return newstring

In [93]:
def getUniqueIngredients(list):
    ingredientList = []
    for data in list:
        #print("Data: "+data)
        for ingredient in str(data).split(","):
            #print("Get ingredient: "+ingredient)
            ingredient = ingredient.strip()
            ingredientList.append(ingredient)
    #newingredientList = list(dict.fromkeys(ingredientList))
    return ingredientList

In [94]:
def createIngredientTTL(ingredients, path):

    noOwlSameAs = """halali:{slug} a foodlirmm:Ingredient;
   rdfs:label "{name}" .

   """

    withOwlSameAs = """halali:{slug} a foodlirmm:Ingredient;
   rdfs:label "{name}";
   owl:sameAs dbr:{slug} .

   """
    for ingredient in ingredients:
        print("Creating ingredient: " + str(ingredient))
        slugIngredient = slugify.slugify(ingredient)
        slugIngredient = str(slugIngredient).replace("-", " ")
        slugIngredient = slugIngredient.title()
        slugIngredient = slugIngredient.replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","")
    # url = "https://dbpedia.org/sparql?default-graph-uri=http%3A%2F%2Fdbpedia.org&query=ASK+%7B%0D%0A+%3Chttp%3A%2F%2Fdbpedia.org%2Fresource%2F%7Bslug%7D%3E+%3Fp+%3Fo+.%0D%0A%7D&format=text%2Fhtml&CXML_redir_for_subjs=121&CXML_redir_for_hrefs=&timeout=30000&debug=on&run=+Run+Query+%22.format(  
    #     slug=slugIngredient)
    # r = requests.get(url)
    # data = r.text
    # if "true" in data:
    #     print("Get owl:sameAs")
    #     withOwlSameAs = withOwlSameAs.format(name=ingredient, slug=slugIngredient)
    #     f = open("data\\dat\\new\\jadi\\noprefix\\ingredients\\" + slugIngredient + ".ttl", "a+", encoding="utf-8")
    #     f.write(withOwlSameAs)
    #     f.close()
    #     withOwlSameAsPrefix = prefix + withOwlSameAs
    #     f = open("data\\dat\\new\\jadi\\withprefix\\ingredients\\" + slugIngredient + ".ttl", "a+", encoding="utf-8")
    #     f.write(withOwlSameAsPrefix)
    #     f.close()
    # else:
    #     print("Doesn't get owl:sameAs")
    #     noOwlSameAs = noOwlSameAs.format(name=ingredient, slug=slugIngredient)
    #     f = open("data\\dat\\new\\jadi\\noprefix\\ingredients\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    #     f.write(noOwlSameAs)
    #     f.close()
    #     noOwlSameAsPrefix = prefix + noOwlSameAs
    #     f = open("data\\dat\\new\\jadi\\withprefix\\ingredients\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    #     f.write(noOwlSameAsPrefix)
    #     f.close()

        pathFolderNoPrefix = path + "\\resources\\noprefix\\ingredients"
        pathFolderWithPrefix = path + "\\resources\\withprefix\\ingredients"

        if not os.path.isdir(pathFolderNoPrefix):
            os.makedirs(pathFolderNoPrefix)

        if not os.path.isdir(pathFolderWithPrefix):
            os.makedirs(pathFolderWithPrefix)

        # print("Doesn't get owl:sameAs")
        noOwlSameAs = noOwlSameAs.format(name=ingredient, slug=slugIngredient)
        f = open(pathFolderNoPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
        f.write(noOwlSameAs)
        f.close()
        noOwlSameAsPrefix = prefix + noOwlSameAs
        f = open(pathFolderWithPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
        f.write(noOwlSameAsPrefix)
        f.close()

In [95]:
def createIngredientSeparate(ingredient, path):
    ingText = """halali:{sluging} a foodlirmm:Ingredient;
   rdfs:label "{nameing}" .

   """
    
    ingredient = str(ingredient).replace('"','').replace("?","").replace("$","").replace("(e)","").replace("(","").replace(")","").replace("%","").replace(".","").replace("\\","").replace("&","")
    print("Creating ingredient: " + str(ingredient))
    slugIngredient = slugify.slugify(ingredient)
    slugIngredient = str(slugIngredient).replace("-", " ")
    slugIngredient = slugIngredient.title()
    slugIngredient = slugIngredient.replace(" ", "_").replace("%","").replace("$","").replace("(","").replace(")","")
        

    pathFolderNoPrefix = path + "\\resources\\noprefix\\ingredients"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\ingredients"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

        # print("Doesn't get owl:sameAs")
        #noOwlSameAs = noOwlSameAs.format(name=ingredient, slug=slugIngredient)
    ingText = ingText.format(sluging=slugIngredient, nameing=ingredient)
    f = open(pathFolderNoPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    f.write(ingText)
    f.close()
    noOwlSameAsPrefix = prefix + ingText
    f = open(pathFolderWithPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    f.write(noOwlSameAsPrefix)
    f.close()

In [96]:
def createManufactureTTL(manufacture, path):
    manufactureText = """halalm:{slug} a foaf:Organization;
		rdfs:label "{name}" .

		"""
    slugManufacture = slugify.slugify(manufacture)
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")
    manufacture = str(manufacture).replace('"','').replace("?","").replace("$","").replace("(","").replace(")","").replace("\\","").replace("&","")
    manufactureText = manufactureText.format(slug=slugManufacture, name=manufacture)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\manufactures"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\manufactures"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    f = open(pathFolderNoPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureText)
    f.close()
    manufactureTextPrefix = prefix + manufactureText
    f = open(pathFolderWithPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureTextPrefix)
    f.close()

In [97]:
def createManufactureTTLSusulan(manufacture, path):
    manufactureText = """halalm:{slug} a foaf:Organization;
		rdfs:label "{name}" .

		"""
    slugManufacture = slugify.slugify(manufacture)
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")
    manufacture = str(manufacture).replace('"','').replace("?","").replace("$","").replace("(","").replace(")","").replace("\\","").replace("&","")
    manufactureText = manufactureText.format(slug=slugManufacture, name=manufacture)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\manufactures"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\manufactures"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    f = open(pathFolderNoPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureText)
    f.close()
    manufactureTextPrefix = prefix + manufactureText
    f = open(pathFolderWithPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureTextPrefix)
    f.close()

In [98]:
def createCertificateTTL(row, path):
    certString = """halalc:{slugCertificate} a halalv:HalalCertificate;
			halalv:halalCode "{certificate}";
			halalv:halalExp "{expired}"^^xsd:date;
			halalv:OrgCert halals:{slugOrg} .

			"""

    orgString = """halals:{slugOrganization} a foaf:Organization;
    rdfs:label "{orgName}";
    foaf:homepage "{cert_bodyname}";
    foaf:based_near {location} .

    """

    expired = formatDateX(str(row['expired']))

    certCode = str(row['cert_code'])
    if ('0' == str(row['cert_code'])):
        certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
    elif ('nan' == str(row['cert_code'])):
        certCode = str(row['cert_bodyname']).upper()
    else:
        certCode = str(row['cert_code']).upper()

    slugCertificate = slugify.slugify(certCode)
    slugCertificate = str(slugCertificate).replace("-", "_")
    slugCertificate = slugCertificate.upper()

    slugOrganization = slugify.slugify(str(row['cert_bodyname']))
    slugOrganization = str(slugOrganization).replace("-", " ")
    slugOrganization = slugOrganization.title()
    slugOrganization = str(slugOrganization).replace(" ", "_")

    certString = certString.format(slugCertificate=slugCertificate, certificate=str(row['cert_code']),
                                   expired=str(expired), slugOrg=slugOrganization)
    orgString = orgString.format(slugOrganization=slugOrganization, orgName=str(row['cert_bodyname']),
                                 cert_bodyname=str(row['homepage']), location=str(row['location']))

    pathFolderNoPrefix = path + "\\resources\\noprefix\\certificates"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\certificates"
    pathFolderNoPrefixOrg = path + "\\resources\\noprefix\\halalsources"
    pathFolderWithPrefixOrg = path + "\\resources\\withprefix\\halalsources"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    if not os.path.isdir(pathFolderNoPrefixOrg):
        os.makedirs(pathFolderNoPrefixOrg)

    if not os.path.isdir(pathFolderWithPrefixOrg):
        os.makedirs(pathFolderWithPrefixOrg)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + slugCertificate
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + slugCertificate
    pathFileNoPrefixOrg = pathFolderNoPrefixOrg + "\\" + slugOrganization
    pathFileWithPrefixOrg = pathFolderWithPrefixOrg + "\\" + slugOrganization

    f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certString)
    f.close()
    certStringPrefix = prefix + certString
    f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certStringPrefix)
    f.close()

    f = open(pathFileNoPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgString)
    f.close()
    orgStringPrefix = prefix + orgString
    f = open(pathFileWithPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgStringPrefix)
    f.close()

In [99]:
def createCertificateTTLSusulan(row, path):
    certString = """halalc:{slugCertificate} a halalv:HalalCertificate;
			halalv:halalCode "{certificate}";
			halalv:halalExp "{expired}"^^xsd:date;
			halalv:OrgCert halals:{slugOrg};
			halalv:halalStatus {status} .

			"""

    orgString = """halals:{slugOrganization} a foaf:Organization;
    rdfs:label "{orgName}";
    foaf:homepage "{cert_bodyname}";
    foaf:based_near {location} .

    """

    expired = formatDateX(str(row['expired']))

    certCode = str(row['cert_code'])
    if ('0' == str(row['cert_code'])):
        certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
    elif ('nan' == str(row['cert_code'])):
        certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
    else:
        certCode = str(row['cert_code']).upper()

    certStatus = "halalv:"+str(row['status'])

    slugCertificate = slugify.slugify(certCode)
    slugCertificate = str(slugCertificate).replace("-", "_")
    slugCertificate = slugCertificate.upper()

    slugOrganization = slugify.slugify(str(row['cert_bodyname']))
    slugOrganization = str(slugOrganization).replace("-", " ")
    slugOrganization = slugOrganization.title()
    slugOrganization = str(slugOrganization).replace(" ", "_")

    certString = certString.format(slugCertificate=slugCertificate, certificate=str(row['cert_code']),
                                   expired=str(expired), slugOrg=slugOrganization, status=certStatus)
    orgString = orgString.format(slugOrganization=slugOrganization, orgName=str(row['cert_bodyname']),
                                 cert_bodyname=str(row['homepage']), location=str(row['location']))

    pathFolderNoPrefix = path + "\\resources\\noprefix\\certificates"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\certificates"
    pathFolderNoPrefixOrg = path + "\\resources\\noprefix\\halalsources"
    pathFolderWithPrefixOrg = path + "\\resources\\withprefix\\halalsources"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    if not os.path.isdir(pathFolderNoPrefixOrg):
        os.makedirs(pathFolderNoPrefixOrg)

    if not os.path.isdir(pathFolderWithPrefixOrg):
        os.makedirs(pathFolderWithPrefixOrg)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + slugCertificate
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + slugCertificate
    pathFileNoPrefixOrg = pathFolderNoPrefixOrg + "\\" + slugOrganization
    pathFileWithPrefixOrg = pathFolderWithPrefixOrg + "\\" + slugOrganization

    f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certString)
    f.close()
    certStringPrefix = prefix + certString
    f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certStringPrefix)
    f.close()

    f = open(pathFileNoPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgString)
    f.close()
    orgStringPrefix = prefix + orgString
    f = open(pathFileWithPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgStringPrefix)
    f.close()

In [100]:
def createProductTTL(row, path):
    productString = """halalf:{slugName} a foodlirmm:FoodProduct;
	a food:Food;
	foodlirmm:code "{code}";
	rdfs:label "{name}";
	gr:hasManufacturer halalm:{slugManufacture} .

halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

"""

    productStringWithoutCertificate = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .

    halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

    halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

    """

    productStringNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .

    halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

    """

    productStringWithoutCertificateNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
        	a food:Food;
        	foodlirmm:code "{code}";
        	rdfs:label "{name}";
        	gr:hasManufacturer halalm:{slugManufacture} .

        """

    product = str(row['name'])
    product = str(product).strip()
    product = product.replace('"', '').replace('\\','')
    productSlug = makeSlug(product)

    productCode = str(row['code'])

    if productCode == "nan":
        productCode = "0000"

    slugManufacture = slugify.slugify(str(row['manufacture']))
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")

    ingredientEntities = str(row['ingredients']).strip()
    prefixEntity = "halali:"
    stringingredientsEntities = ingredientEntities.replace(", ", ",halali:").replace(".", "").replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","").replace("-","").replace("=","").replace("'","").replace("&","")
    finalStringIngredientEntities = prefixEntity + stringingredientsEntities

    prefixString = "\""
    stringIngredients = ingredientEntities.replace(", ", "\", \"")
    finalStringIngredientStrings = prefixString + stringIngredients + prefixString

    certbody_name = str(row['cert_bodyname'])
    if certbody_name == "nan" and ingredientEntities == "nan":
        productString = productStringWithoutCertificateNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                                               slugManufacture=slugManufacture)
    elif certbody_name != "nan" and ingredientEntities != "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productString.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate,
                                             concateIngredientsEntity=finalStringIngredientEntities,
                                             concateIngredientsString=finalStringIngredientStrings)
    elif certbody_name != 'nan' and ingredientEntities == "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productStringNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate)
    elif certbody_name == 'nan' and ingredientEntities != "nan":
        productString = productStringWithoutCertificate.format(slugName=productSlug, code=productCode,
                                                                           name=product,
                                                                           slugManufacture=slugManufacture,
                                                                           concateIngredientsEntity=finalStringIngredientEntities,
                                                                           concateIngredientsString=finalStringIngredientStrings)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\foodproducts"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\foodproducts"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + productSlug
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + productSlug
    if not os.path.exists(pathFileNoPrefix):
        f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()
    else:
        f = open(pathFileNoPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()

    if not os.path.exists(pathFileWithPrefix):
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()
    else:
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()

In [101]:
def createProductTTLSusulan(row, path):
    productString = """halalf:{slugName} a foodlirmm:FoodProduct;
	a food:Food;
	foodlirmm:code "{code}";
	rdfs:label "{name}";
	gr:hasManufacturer halalm:{slugManufacture} .

halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

"""

    productStringWithoutCertificate = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}".
    

    halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

    halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

    """

    productStringNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}".
    

    halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

    """

    productStringWithoutCertificateNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
        	a food:Food;
        	foodlirmm:code "{code}";
        	rdfs:label "{name}";
        	gr:hasManufacturer halalm:{slugManufacture} .

        """

    product = str(row['name'])
    product = str(product).strip()
    product = product.replace('"', '').replace('\\','')
    productSlug = makeSlug(product)

    productCode = str(row['code'])

    if productCode == "nan":
        productCode = "0000"

    slugManufacture = slugify.slugify(str(row['manufacture']))
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")

    ingredientEntities = str(row['ingredients']).strip()
    prefixEntity = "halali:"
    stringingredientsEntities = ingredientEntities.replace(", ", ",halali:").replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","").replace("-","").replace("=","").replace("'","").replace("&","")
    finalStringIngredientEntities = prefixEntity + stringingredientsEntities

    prefixString = "\""
    stringIngredients = ingredientEntities.replace(", ", "\", \"")
    finalStringIngredientStrings = prefixString + stringIngredients + prefixString

    certbody_name = str(row['cert_bodyname'])
    if certbody_name == "nan" and ingredientEntities == "nan":
        productString = productStringWithoutCertificateNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                                               slugManufacture=slugManufacture)
    elif certbody_name != "nan" and ingredientEntities != "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productString.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate,
                                             concateIngredientsEntity=finalStringIngredientEntities,
                                             concateIngredientsString=finalStringIngredientStrings)
    elif certbody_name != 'nan' and ingredientEntities == "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productStringNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate)
    elif certbody_name == 'nan' and ingredientEntities != "nan":
        productString = productStringWithoutCertificate.format(slugName=productSlug, code=productCode,
                                                                           name=product,
                                                                           slugManufacture=slugManufacture,
                                                                           concateIngredientsEntity=finalStringIngredientEntities,
                                                                           concateIngredientsString=finalStringIngredientStrings)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\foodproducts"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\foodproducts"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + productSlug
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + productSlug
    if not os.path.exists(pathFileNoPrefix):
        f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()
    else:
        f = open(pathFileNoPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()

    if not os.path.exists(pathFileWithPrefix):
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()
    else:
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()

In [102]:
def createProductTTLSusulanCoba(row, path):
    productString = """halalf:{slugName} a foodlirmm:FoodProduct;
	a food:Food;
	foodlirmm:code "{code}";
	rdfs:label "{name}";
	gr:hasManufacturer halalm:{slugManufacture} .
    foodlirmm:certificate halalc:{slugCertificate} .
    food:containsIngredient {concateIngredientsEntity} .
    food:ingredientsListAsText {concateIngredientsString} .

"""

    productStringWithoutCertificate = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .
        food:containsIngredient {concateIngredientsEntity} .
        food:ingredientsListAsText {concateIngredientsString} .

    """

    productStringNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .
        foodlirmm:certificate halalc:{slugCertificate} .

    """

    productStringWithoutCertificateNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
        	a food:Food;
        	foodlirmm:code "{code}";
        	rdfs:label "{name}";
        	gr:hasManufacturer halalm:{slugManufacture} .

        """

    product = str(row['name'])
    product = str(product).strip()
    product = product.replace('"', '').replace('\\','')
    productSlug = makeSlug(product)

    productCode = str(row['code'])

    if productCode == "nan":
        productCode = "0000"

    slugManufacture = slugify.slugify(str(row['manufacture']))
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")

    ingredientEntities = str(row['ingredients']).strip()
    prefixEntity = "halali:"
    stringingredientsEntities = ingredientEntities.replace(", ", ",halali:").replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","").replace("-","").replace("=","").replace("'","").replace("&","")
    finalStringIngredientEntities = prefixEntity + stringingredientsEntities

    prefixString = "\""
    stringIngredients = ingredientEntities.replace(", ", "\", \"")
    finalStringIngredientStrings = prefixString + stringIngredients + prefixString

    certbody_name = str(row['cert_bodyname'])
    if certbody_name == "nan" and ingredientEntities == "nan":
        productString = productStringWithoutCertificateNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                                               slugManufacture=slugManufacture)
    elif certbody_name != "nan" and ingredientEntities != "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productString.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate,
                                             concateIngredientsEntity=finalStringIngredientEntities,
                                             concateIngredientsString=finalStringIngredientStrings)
    elif certbody_name != 'nan' and ingredientEntities == "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productStringNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate)
    elif certbody_name == 'nan' and ingredientEntities != "nan":
        productString = productStringWithoutCertificate.format(slugName=productSlug, code=productCode,
                                                                           name=product,
                                                                           slugManufacture=slugManufacture,
                                                                           concateIngredientsEntity=finalStringIngredientEntities,
                                                                           concateIngredientsString=finalStringIngredientStrings)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\foodproducts"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\foodproducts"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + productSlug
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + productSlug
    if not os.path.exists(pathFileNoPrefix):
        f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()
    else:
        f = open(pathFileNoPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()

    if not os.path.exists(pathFileWithPrefix):
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()
    else:
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()

In [103]:
def execCreateProductTTL(df, path):
    for index, row in df.iterrows():
        print("Creating product ttl on: " + str(row['name']))
        if str(row['name']) != "nan":
            createProductTTL(row, path)

In [104]:
def execCreateProductTTLSusulan(df, path):
    for index, row in df.iterrows():
        print("Creating product ttl on: " + str(row['name']))
        if str(row['name']) != "nan":
            createProductTTLSusulan(row, path)

In [105]:
def execCreateProductTTLSusulanCoba(df, path):
    for index, row in df.iterrows():
        print("Creating product ttl on: " + str(row['name']))
        if str(row['name']) != "nan":
            createProductTTLSusulanCoba(row, path)

In [106]:
def countUniqueValuesEachEntity(df):
    print("Unique values: " + str(len(df['name'].unique().tolist())))

    # Select all duplicate rows based on one column
    duplicateRowsDF = df[df.duplicated(['name'])]
    duplicateRowsDF = duplicateRowsDF.sort_values(by=['name'])
    print("Duplicate rows length: " + str(len(duplicateRowsDF)))
    for index, row in duplicateRowsDF.iterrows():
        print("Values are: " + str(row['code']) + " - " + str(row['name']) + " - " + str(
            row['manufacture']) + " - " + str(row['cert_code']) + " - " + str(row['expired']) + " - " + str(
            row['cert_bodies']) + " - " + str(row['cert_bodyname']) + " - " + str(row['homepage']) + " - " + str(
            row['location']) + " - " + str(row['ingredients']))

In [107]:
def execCreateCertAndOrganizationTTL(df, path):
    dfcert = df[['cert_code', 'expired', 'cert_bodies', 'cert_bodyname', 'homepage', 'location']].copy()
    # dfcert = dfcert.dropna()
    # #
    # # df = df.drop_duplicates()
    #
    # for name in dfcert.cert_bodyname.unique():
    #     print("Values is: "+str(name))
    for index, row in dfcert.iterrows():
        if 'nan' == str(row['cert_bodies']):
            print("Skipping, no cert bodyname..")
            pass
        else:
            print("Creating certificate on: " + str(row['cert_code']) + str(row['cert_bodies']))
            createCertificateTTL(row, path)

In [108]:
def execCreateCertAndOrganizationTTLSusulan(df, path):
    dfcert = df[['cert_code', 'expired', 'cert_bodies', 'cert_bodyname', 'homepage', 'location', 'status']].copy()
    # dfcert = dfcert.dropna()
    # #
    # # df = df.drop_duplicates()
    #
    # for name in dfcert.cert_bodyname.unique():
    #     print("Values is: "+str(name))
    for index, row in dfcert.iterrows():
        #if 'nan' == str(row['cert_bodies']):
            #print("Skipping, no cert bodyname..")
            #pass
        #else:
            print("Creating certificate on: " + str(row['cert_code']) )##+ str(row['cert_bodies']))
            createCertificateTTLSusulan(row, path)

In [109]:
def execCreateIngredientsTTLFromFile(df, path):
    with open('ingredients.txt', 'r') as f:
        lines = f.readlines()
        for ingredient in lines:
            ingredient =  ingredient.replace("\n","")
            print("Working on: "+ingredient)
            createIngredientTTL(ingredient, path)

In [110]:
def execCreateIngredientsTTLFromDF(df, path):
    # create ingredients
    listIngredient = df['ingredients'].dropna().tolist()
    ingredients = getUniqueIngredients(listIngredient)
    newlistIng = list(dict.fromkeys(ingredients))
    createIngredientTTL(ingredients, path)

In [111]:
def execCreateIngredientsTTLFromDFCoba(df, path):
    # create ingredients
    listIngredient = df['ingredients'].dropna().tolist()
    ingredients = getUniqueIngredients(listIngredient)
    newlistIng = list(dict.fromkeys(ingredients))
    for ingredient in newlistIng:
        createIngredientSeparate(ingredient, path)

In [112]:
def execCreateManufacturerTTL(df, path):
    df = df.sort_values(by ='manufacture' )
    dfObj = df['manufacture']
    for manufacture in dfObj.unique().tolist():

        if (str(manufacture) == 'nan'):
            pass
        else:
            print("Working on: " + str(manufacture))
            createManufactureTTL(manufacture, path)

In [113]:
def execCreateManufacturerTTLSusulan(df, path):
    df = df.sort_values(by ='manufacture' )
    dfObj = df['manufacture']
    for manufacture in dfObj.unique().tolist():

        if (str(manufacture) == 'nan'):
            pass
        else:
            print("Working on: " + str(manufacture))
            createManufactureTTLSusulan(manufacture, path)

In [114]:
# url = "https://dbpedia.org/sparql?default-graph-uri=http%3A%2F%2Fdbpedia.org&query=ASK+%7B%0D%0A+%3Chttp%3A%2F%2Fdbpedia.org%2Fresource%2F{slug}%3E+%3Fp+%3Fo+.%0D%0A%7D&format=text%2Fhtml&CXML_redir_for_subjs=121&CXML_redir_for_hrefs=&timeout=30000&debug=on&run=+Run+Query+".format(
#     slug="Monosodium_glutamate")
# r = requests.get(url)
# data = r.text
# print("The content is: "+data)
# if "true" in data:
#     print("Get owl:sameAs")
# else:
#     print("Doesn't get owl:sameAs")

# slugIngredient = slugify.slugify("aku apa")
# print(slugIngredient)

#     createIngredientTTL()
# df['manufacture'] = df['manufacture'].apply(lambda x: getTrimAndTitle(x))
# df['name'] = df['name'].apply(lambda x: getTrimAndTitle(x))
# dforgs = df[df['cert_bodyname'] == numpy.nan]
# for dforg in dforgs:
#     print("Value of nan: "+str(len(dforg)))
# print("Size before drop mui: " + str(len(df)) + " - size unique org cert: " + str(len(df.cert_bodyname.unique())))
# df = df[~df['cert_bodyname'].isin(['https://www.halalmui.org/'])]
# print("Size after: " + str(len(df)) + " - size unique omanufacture: "+str(len(df.manufacture.unique()))+" - name "+str(len(df.name.unique())))
# for sert in df.manufacture.unique():
#     print("Sert: "+str(sert))

# for index, row in df.iterrows():
#     print("Value is: name: "+str(row['name'])+" - manufacture: "+str(row['manufacture']))

# df = pd.read_csv(path+".csv", delimiter=";", error_bad_lines=False,names=['id', 'code', 'name', 'verify', 'ingredients'])
# newdf = df[['code']].copy()
#
# newdf['name'] = df['name'].apply(lambda x: getProductName(x))
# newdf['manufacture'] = df['name'].apply(lambda x: getManufactureName(x))
# print("Length code: "+str(len(newdf['code']))+" - name: "+str(len(newdf['name']))+" - manufacture: "+str(len(newdf['manufacture'])))
# newdf['ingredientsx'] = ''
# newdf['ingredients1'] = ''
# newdf['ingredients2'] = ''
# newdf['ingredients3'] = ''
# newdf['ingredients4'] = ''
# newdf['ingredients5'] = ''
# newdf['ingredients'] = df['ingredients'].apply(lambda x: getIngredients(x))
# for data in newdf['name']:
#     print(str(data))
# for index, row in df.iterrows():
#     # print(str(row['code'])+" - "+str(row['name'])+" - "+str(row['ingredients']))
#     print(str(row['code'])+" - "+str(row['name'])+" - "+str(row['ingredients']))
# df['ingredients'] = ''
# df['ingredients1'] = ''
# df['ingredients2'] = ''
# df['ingredients3'] = ''
# df['ingredients4'] = ''
# df['ingredients5'] = ''
# df.to_csv(path+"_ready.csv", sep=";", index=None, header=None)

In [119]:
path = "datanew"
filename = "Hasil Salah5"
df = pd.read_csv(path + "\\" + filename + ".csv", delimiter=";", error_bad_lines=False, low_memory=False)
df
# ,names=['code', 'name', 'manufacture', 'cert_code', 'expired', 'cert_bodies', 'cert_bodyname', 'homepage', 'location', 'ingredients', 'status']
     

,code,name,produkmui,nilai,label,ingredients,ing0,ing1,ing2,ing3,...,ing27,ing28,ing29,ing30,ing31,ing32,cert_bodies,expired,homepage,location
0,19,Qtela Keripik Singkong Balado,Keripik singkong balado ante,6,0,"singkong,minyak kelapa sawit,antioksidan tbhq,...",singkong,minyak kelapasawit antioksidan tbhq,amonium bikarbonat,bumbu rasa balado penguat rasa mononatrium gl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20,Qtela Keripik Singkong Balado,Keripik singkong balado ante,6,0,"singkong,minyak kelapa sawit,antioksidan tbhq,...",singkong,minyak kelapa sawit antioksidan tbhq,amonium bikarbonat,bumbu rasa balado penguat rasa mononatrium gl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21,Qtela Keripik Tempe Original,Keripik tempe original,6,0,"tempe,minyak kelapa sawit antioksidan tbhq,tep...",tempe,minyak kelapa sawit antioksidan tbhq,tepung beras,tapioka,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22,Kusuka Keripik Singkong Balado,Keripik singkong balado ante,6,0,"singkong,minyak kelapa sawit,gula,bumbu rasa b...",singkong,minyak kelapa sawit,gula,bumbu rasa balado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23,Karuhun Keripik Singkong Original,Kita-kita keripik singkong original,6,0,"keripik singkong piliihan,daun jeruk,garam,cab...",keripik singkong piliihan,daun jeruk,garam,cabai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,1276,Pro Chiz Keju Slice 5'S,Keju spesial,"1,42857E+16",0,kosong,kosong,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
703,1277,Pro Chiz Keju Slice 10'S,Keju spesial,"1,42857E+16",0,"water,cheddar cheese,vegetable oil,thickener a...",water,cheddar cheese,vegetable oil,thickener acid treated starch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
704,1278,Tango Drink Don Pedro,Beng beng drink,"1,42857E+16",0,kosong,kosong,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
705,1279,Milo Healthy Drink Polybag Activ-Go,Dr milo,"1,42857E+16",0,"sari malt,barli,gula,susu bubuk skim,maltodeks...",sari malt barli,gula,susu bubuk skim,maltodekstrin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
NaN = np.nan
df["cert_bodies"] = NaN
df["manufacture"] = NaN
df["cert_code"] = NaN
df["expired"] = NaN
df["cert_bodyname"] = NaN
df["homepage"] = NaN
df["location"] = NaN
df["status"] = NaN
df["ingredients"] = df["ingredients"].replace('kosong', np.nan , regex=True)
df["ingredients"] = df["ingredients"].replace(',', ', ' , regex=True)
#df["ingredients"] = df["ingredients"].replace('.', '' , regex=True)
df["name"] = df["name"].str.lower()
df["ingredients"] = df["ingredients"].str.lower()
##df = df[df.label == 0]
##df
##df.to_csv("Hasil Salah.csv")
df

,code,name,produkmui,nilai,label,ingredients,ing0,ing1,ing2,ing3,...,ing31,ing32,cert_bodies,expired,homepage,location,manufacture,cert_code,cert_bodyname,status
0,19,qtela keripik singkong balado,Keripik singkong balado ante,6,0,"singkong, minyak kelapa sawit, antioksidan tbh...",singkong,minyak kelapasawit antioksidan tbhq,amonium bikarbonat,bumbu rasa balado penguat rasa mononatrium gl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20,qtela keripik singkong balado,Keripik singkong balado ante,6,0,"singkong, minyak kelapa sawit, antioksidan tbh...",singkong,minyak kelapa sawit antioksidan tbhq,amonium bikarbonat,bumbu rasa balado penguat rasa mononatrium gl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21,qtela keripik tempe original,Keripik tempe original,6,0,"tempe, minyak kelapa sawit antioksidan tbhq, t...",tempe,minyak kelapa sawit antioksidan tbhq,tepung beras,tapioka,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22,kusuka keripik singkong balado,Keripik singkong balado ante,6,0,"singkong, minyak kelapa sawit, gula, bumbu ras...",singkong,minyak kelapa sawit,gula,bumbu rasa balado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23,karuhun keripik singkong original,Kita-kita keripik singkong original,6,0,"keripik singkong piliihan, daun jeruk, garam, ...",keripik singkong piliihan,daun jeruk,garam,cabai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,1276,pro chiz keju slice 5's,Keju spesial,"1,42857E+16",0,NaN,kosong,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
703,1277,pro chiz keju slice 10's,Keju spesial,"1,42857E+16",0,"water, cheddar cheese, vegetable oil, thickene...",water,cheddar cheese,vegetable oil,thickener acid treated starch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
704,1278,tango drink don pedro,Beng beng drink,"1,42857E+16",0,NaN,kosong,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
705,1279,milo healthy drink polybag activ-go,Dr milo,"1,42857E+16",0,"sari malt, barli, gula, susu bubuk skim, malto...",sari malt barli,gula,susu bubuk skim,maltodekstrin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
execCreateProductTTLSusulan(df, path)
execCreateCertAndOrganizationTTLSusulan(df, path)
execCreateManufacturerTTLSusulan(df, path)
execCreateIngredientsTTLFromDFCoba(df, path)

Creating product ttl on: qtela keripik singkong balado
Creating product ttl on: qtela keripik singkong balado
Creating product ttl on: qtela keripik tempe original
Creating product ttl on: kusuka keripik singkong balado
Creating product ttl on: karuhun keripik singkong original
Creating product ttl on: kusuka keripik singkong original
Creating product ttl on: indomaret keripik pisang madu
Creating product ttl on: indomaret snack kue tambang
Creating product ttl on: qtela keripik singkong original
Creating product ttl on: qtela keripik singkong original
Creating product ttl on: luwak kopi bubuk murni
Creating product ttl on: nescafe coffee cream
Creating product ttl on: golda coffee drink dolce latte
Creating product ttl on: nestle air mineral pure life
Creating product ttl on: x'tragin minuman jahe merah
Creating product ttl on: kayarasa jahe merah instant
Creating product ttl on: karuhun keripik singkong original sp
Creating product ttl on: lovy butter cookies choco butter
Creating pr

Creating product ttl on: cheetos snack jagung bakar
Creating product ttl on: cheetos snack jagung bakar
Creating product ttl on: mogu-moguminuman nata de coco strawberry
Creating product ttl on: panda grass jelly drink
Creating product ttl on: skippy jam creamy peanut butter
Creating product ttl on: kusuka keripik singkong keju bakar
Creating product ttl on: aqua galon kosong + isi ulang
Creating product ttl on: sukses's mie goreng isi 2 ayam kremes
Creating product ttl on: indomie mi instan soto mie
Creating product ttl on: indomie mi instan ayam bawang
Creating product ttl on: hatari crackers see hong puff sugar
Creating product ttl on: cap enaak kental manis coklat
Creating product ttl on: olivoila olive oil extra virgin
Creating product ttl on: maya sardens tomato,chili sauce
Creating product ttl on: torabika cappuccino extra choco granule
Creating product ttl on: good day kopi instant 3 in 1 mocacinno
Creating product ttl on: gadjah kopi tubruk manis 2 in 1
Creating product ttl on

Creating product ttl on: lo han kuo minuman infusion
Creating product ttl on: lay's snack potato chips bbq fiesta
Creating product ttl on: happytos snack tortilla chips roasted corn
Creating product ttl on: milo healthy drink 3 in 1 actigen-e
Creating product ttl on: indofood sambal ala bangkok asam manis
Creating product ttl on: morinaga chil school susu pertumbuhan vanila
Creating product ttl on: happytos snack tortilla chips jagung bakar
Creating product ttl on: oishi snack kraker u,g pedas
Creating product ttl on: swiss miss dark chocolate drink
Creating product ttl on: kapal api krim kafe non dairy creamer
Creating product ttl on: aik cheong white coffee original
Creating product ttl on: khong guan wafer stick chocolate
Creating product ttl on: nabati bisvit marie sandwich cheese cream
Creating product ttl on: garuda snack potato daging sapi bbq
Creating product ttl on: alpenliebe candy chocolate caramel eclairs
Creating product ttl on: loacker wafer quadratini cappuccino
Creating

Creating product ttl on: energen cereal,susu instant vanila
Creating product ttl on: milo cereal balls combo pack coklat
Creating product ttl on: mr.hottest snack tortilla chips maitos sambal balado
Creating product ttl on: buavita juice slim lychee
Creating product ttl on: potabee snack potato chips ayam bakar
Creating product ttl on: mujigae topokki original
Creating product ttl on: nissin mie goreng gekikara ramen premium
Creating product ttl on: chitato snack potato chips sapi bumbu bakar
Creating product ttl on: nissin mie instant gekikara ramen pedas
Creating product ttl on: meiji plain crackers
Creating product ttl on: a.t.b marie milk
Creating product ttl on: indomaret instant cup noodle ayam pedas
Creating product ttl on: bernardi corned beef
Creating product ttl on: biskuat biscuit energi coklat
Creating product ttl on: indomaret kuaci susu
Creating product ttl on: oatbits biskuit oat raisin kismis
Creating product ttl on: mister potato snack crisps roasted beef
Creating prod

Creating product ttl on: buavita apple
Creating product ttl on: indomaret snack pang-pang
Creating product ttl on: greenday for kids crispy broccoli
Creating product ttl on: nissin walens blueberry soes pck
Creating product ttl on: indomaret kuaci
Creating product ttl on: weston biscuit bagelan butter kecil
Creating product ttl on: lotus biscoff biscuit karamel
Creating product ttl on: kraft sandwich oreo strawbery creme
Creating product ttl on: mama mie instant noodles kimchi
Creating product ttl on: tango drink creamio banana pudding
Creating product ttl on: loacker wafer quadratini cocoa,milk
Creating product ttl on: mizone move on minuman isotonik starfruit
Creating product ttl on: mizone mood up minuman isotonik cranberry
Creating product ttl on: picnic dodol garut
Creating product ttl on: inaco jelly mini,15's
Creating product ttl on: lay's snack potato chips saus krim,onion
Creating product ttl on: indomaret kemplang bulat
Creating product ttl on: chrysanthemum granule
Creating 

...

Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certificate on: nan
Creating certi

Creating ingredient: pengental nabati
Creating ingredient: pewarta
Creating ingredient: tartrazin ci
Creating ingredient: bubuk bawang merah
Creating ingredient: perisa sintetik ayam
Creating ingredient: ektrak ayam bubuk cabai
Creating ingredient: kecap manis
Creating ingredient: ikan sarden
Creating ingredient: modified starch
Creating ingredient: bumbu rasa rumput laut
Creating ingredient: tuna
Creating ingredient: kecap
Creating ingredient: tepung kentang
Creating ingredient: perisa sapi panggang
Creating ingredient: aspartam
Creating ingredient: amonia karamel kalium klorida
Creating ingredient: lemak reroti
Creating ingredient: askorbil palamitat
Creating ingredient: tokoferol
Creating ingredient: susu bubuk full cream
Creating ingredient: dekstrosa
Creating ingredient: pengemulsi lesitin kedelai
Creating ingredient: pengembang natrium bikarbonat perisa artifisial vanili
Creating ingredient: g
Creating ingredient: antioksidan bha
Creating ingredient: whey bubuk
Creating ingredien

Creating ingredient: premiks mineral
Creating ingredient: daun pan
Creating ingredient: saos sambal
Creating ingredient: gula pemanis buatan
Creating ingredient: bulir jeruk
Creating ingredient: konsentrat jeruk
Creating ingredient: perisa jeruk
Creating ingredient: pewarna natural
Creating ingredient: kelapa
Creating ingredient: gula tebu
Creating ingredient: sirup singkong
Creating ingredient: perisa vanili
Creating ingredient: tapioca flour
Creating ingredient: rice flour
Creating ingredient: flavour enhancer
Creating ingredient: mononatrium l-glutamat
Creating ingredient: dinatrium ' - inosinat
Creating ingredient: dinatrium ' - guanilat
Creating ingredient: white pepper powder
Creating ingredient: nutrneg essential oil
Creating ingredient: gula aren
Creating ingredient: hidrosilat protein nabati
Creating ingredient: bubul bawang merah
Creating ingredient: bubuk jamur
Creating ingredient: bubuk lada putih
Creating ingredient: bubuk lada hitam
Creating ingredient: bubuk jahe
Creatin

Creating ingredient: ekstrak stroberi
Creating ingredient: pasta cokelat
Creating ingredient: kacang tanah panggang
Creating ingredient: minyak kacang tanah
Creating ingredient: perisa sintetik susu
Creating ingredient: susu bubuk lesitin
Creating ingredient: cokelat pasta
Creating ingredient: mentega asli
Creating ingredient: lemak tumbuh-tumbuhan
Creating ingredient: kuning telur
Creating ingredient: cuka
Creating ingredient: mustard
Creating ingredient: gom xantan
Creating ingredient: sekuestran
Creating ingredient: kalsium dinatrium edta kadar lemak
Creating ingredient: pewarna makana kuning fcf ci
Creating ingredient: kopi mikro
Creating ingredient: cokelat paduan putih
Creating ingredient: lemak
Creating ingredient: sirup glukosa bubuk
Creating ingredient: pewarna alami merah bit
Creating ingredient: fos
Creating ingredient: ekstrak air kelapa muda
Creating ingredient: perisa artifisial kelapa muda
Creating ingredient: minyak inti kelapa sawit terhidrogenasi
Creating ingredient: 

Creating ingredient: malt
Creating ingredient: biru ci
Creating ingredient: perisa moka
Creating ingredient: eritrosin ci
Creating ingredient: tartrasin ci
Creating ingredient: kacang-kacangan
Creating ingredient: cokelat compound
Creating ingredient: pgpr
Creating ingredient: pati termodifikasi penstabil
Creating ingredient: gelatin sapi
Creating ingredient: monokalsium fosfat
Creating ingredient: protein susu terhidrolisa
Creating ingredient: perisa identik alami almond antioksidan tokoferol
Creating ingredient: entrosin ci
Creating ingredient: terbuat dari oat
Creating ingredient: kentang asli
Creating ingredient: pengemulsi lesitin kedelai monosodium glutamat
Creating ingredient: pewarna makanan tartrazine ci
Creating ingredient: jahe merah
Creating ingredient: collagen
Creating ingredient: ekstrak sereh
Creating ingredient: ekstrak kulit manggis
Creating ingredient: tehhijau
Creating ingredient: konsentrat stroberi
Creating ingredient: saus berperisa sayuran
Creating ingredient: p